In [1]:
import os
import cv2
import numpy as np
import imutils
import random
import time
import sys
import json

from PIL import ImageFont, ImageDraw, Image  

DATA_DIRECTORY = "C:/Users/Sander/Desktop/data"
TEST_OUTPUT = "C:/Users/Sander/Desktop/"

In [2]:
# Adds text to a desktop shortcut image (below the icon)
def addShortcutText(image, text, font, icon_within_border, icon_text_max_height, icon_image_max_height):
    # Convert the image to RGB (OpenCV uses BGR)  
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA)  

    # Pass the image to PIL  
    image_pil = Image.fromarray(image_rgb)  
    draw = ImageDraw.Draw(image_pil)  

    # use a truetype font  
    w, _ = draw.textsize(text, font=font)
    _, h = draw.textsize("h", font=font)

    # Get text coordinates
    textX = int((image.shape[1] - w) / 2)
    textY = int((icon_text_max_height - h) / 2) + icon_within_border
    
    # Draw the text
    draw.text((textX, textY + icon_image_max_height), text, font=font, fill=(255, 255, 255, 255))
    
    # Get back the image to OpenCV  
    image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGBA2BGRA)
    
    return image

In [3]:
# Adds text to a given background image
def createTextField(image, text, font):
    # Additional space to left and right of text
    left_border = 0
    right_border = 0
    
    # Make sure background image is large enough in terms of width
    while image.shape[1] < 500:
        image = np.hstack((image, image))
    
    # Convert the image to RGB (OpenCV uses BGR)  
    textImage_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  

    # Pass the image to PIL  
    textImage_pil = Image.fromarray(textImage_rgb)  
    draw = ImageDraw.Draw(textImage_pil)  

    # use a truetype font  
    w, _ = draw.textsize(text, font=font)
    _, h = draw.textsize("h", font=font)

    # Get text coordinates
    textX = left_border
    textY = int((image.shape[0] - h) / 2) - 1
    
    # Draw the text
    draw.text((textX, textY), text, font=font, fill=(0, 0, 0, 255))

    # Get back the image to OpenCV  
    textImage = cv2.cvtColor(np.array(textImage_pil), cv2.COLOR_RGB2BGR)
    
    # Fit image to text size (remove remaining background)
    textImage = textImage[0:textImage.shape[1], 0:w+left_border+right_border]
    
    return textImage

In [4]:
# Overlays a transparent image onto another transparent image
def insertTransparentImage(image, overlay, x1, x2, y1, y2):
    img = image.copy()
    
    alpha_overlay = overlay[:, :, 3] / 255.0
    alpha_image = 1.0 - alpha_overlay

    # Intersect by alpha-value
    for c in range(0, 3):
        img[y1:y2, x1:x2, c] = (alpha_overlay * overlay[:, :, c] + alpha_image * img[y1:y2, x1:x2, c])
        
    return img

In [5]:
# Adds a border effect to a given image
def addWindowBorderEffect(window):
    # Get border size
    size = border_effect_size
    
    # Make image larger such that border fits
    image = np.zeros((window.shape[0]+size*2, window.shape[1]+size*2, 4))
    
    # Draw border line-by-line
    for i in range(0, size):
        # Get transparency of current line
        transparency = 255 / (2 ** (size-i))
        
        # Get border color
        color = (120, 120, 120, transparency)
        
        # Draw border line
        image[i:i+1, i:image.shape[1]-i] = color
        image[i:image.shape[0]-i, image.shape[1]-i-1:image.shape[1]-i] = color
        image[image.shape[0]-i-1:image.shape[0]-i, i:image.shape[1]-i] = color
        image[i:image.shape[0]-i, i:i+1] = color
    
    # Add window to the drawn border
    image[size:size+window.shape[0], size:size+window.shape[1]] = window
    
    return image

In [6]:
# Add necessary paths
path_to_additional_data = DATA_DIRECTORY + "/windows_files/data/"
path_to_screenshots = DATA_DIRECTORY + "/windows_files/screens/"
path_to_icons = DATA_DIRECTORY + "/icons/"

path_to_wallpapers = DATA_DIRECTORY + "/windows_files/wallpapers/"
path_to_right_taskbar_elements = DATA_DIRECTORY + "/windows_files/taskbar_right/"

path_to_window_elements = DATA_DIRECTORY + "/windows_files/window/"
path_to_websites = DATA_DIRECTORY + "/websites/"

In [7]:
# Load all required images

# Left part of taskbar
taskbar_left_original = cv2.imread(os.path.join(path_to_additional_data, 'taskbar_left.png'))
# Middle part of taskbar
taskbar_middle = cv2.imread(os.path.join(path_to_additional_data, 'taskbar_middle.png'))
# Effect of an "open" task 
taskbar_dot = cv2.imread(os.path.join(path_to_additional_data, 'taskbar_dot.png'), cv2.IMREAD_UNCHANGED)
# Effect of a "currently shown" task
taskbar_selected = cv2.imread(os.path.join(path_to_additional_data, 'taskbar_selected.png'), cv2.IMREAD_UNCHANGED)
# Right part of taskbar
taskbar_right_original = cv2.imread(os.path.join(path_to_additional_data, 'taskbar_right.png'))

# Right part of a window head
window_end = cv2.imread(os.path.join(path_to_window_elements, 'window_end.png'))
# Middle part of a window head
window_filler = cv2.imread(os.path.join(path_to_window_elements, 'window_filler.png'))

# Left part of MS Edge browser head
edge_start = cv2.imread(os.path.join(path_to_window_elements, 'edge_start.png'))
# Middle part of MS Edge browser head
edge_middle = cv2.imread(os.path.join(path_to_window_elements, 'edge_middle.png'))
# Right part of MS Edge browser head
edge_end = cv2.imread(os.path.join(path_to_window_elements, 'edge_end.png'))

# Path to fonts
path_to_font_bold = os.path.join(path_to_additional_data, 'SegoeUI_Bold.ttf')
path_to_font_normal = os.path.join(path_to_additional_data, 'SegoeUI.ttf')

# Mouse icon
mouse_icon = cv2.imread(os.path.join(path_to_additional_data, 'mouse.png'), cv2.IMREAD_UNCHANGED)

# Screenshot size
wallpaper_width = 1920
wallpaper_height = 1080

# Width of border effect for application windows
border_effect_size = 5

# Scale mouse to screenshot size
scale = max(wallpaper_width / 110, 1)
mouse_icon = imutils.resize(mouse_icon, width=int(scale))

# Scale taskbar to screenshot size
scale = wallpaper_height / 24
taskbar_left_original = imutils.resize(taskbar_left_original, height=int(scale))
taskbar_middle = imutils.resize(taskbar_middle, height=int(scale))
taskbar_right_original = imutils.resize(taskbar_right_original, height=int(scale))

# Scale MS Edge browser head to screenshot size
scale = wallpaper_height / 36
edge_start = imutils.resize(edge_start, height=int(scale))
edge_middle = imutils.resize(edge_middle, height=int(scale))
edge_end = imutils.resize(edge_end, height=int(scale))

# Scale window head to screenshot size
scale = wallpaper_height / 45
window_end = imutils.resize(window_end, height=int(scale))
window_filler = imutils.resize(window_filler, height=int(scale))

In [18]:
# Create an empty desktop image
def createDesktop():
    # Copy left and right taskbar ends
    taskbar_left = taskbar_left_original.copy()
    taskbar_right = taskbar_right_original.copy()
    
    # Select a random wallpaper
    if len(os.listdir(path_to_wallpapers)) == 0:
        print('Did not find any wallpapers in ' + path_to_wallpapers)
    wallpaper_file = random.choice(os.listdir(path_to_wallpapers))
    wallpaper = cv2.imread(os.path.join(path_to_wallpapers, wallpaper_file))
    
    # Only scales to width!
    #wallpaper = imutils.resize(wallpaper, width=wallpaper_width)
    wallpaper = cv2.resize(wallpaper, (wallpaper_width, wallpaper_height))
    
    # Randomly decide if we add a search bar to taskbar
    search = random.randint(0, 2)

    if search > 0:
        if search == 1:
            search = "search1.png"
        elif search == 2:
            search = "search2.png"

        # Add searchbar
        search_task = cv2.imread(os.path.join(path_to_additional_data, search))
        search_task = imutils.resize(search_task, height=taskbar_left.shape[0])

        taskbar_left = np.hstack((taskbar_left, search_task))

    # Pick a random number of task icons
    number_of_elements = random.randint(2, 15)
    tasks = random.sample(os.listdir(path_to_icons), k=number_of_elements)

    '''
    # OLD VERSION (without "selected-effect")
    for i in range(0, len(tasks)):
        task = cv2.imread(os.path.join(path_to_left_taskbar_elements, tasks[i]))
        task = imutils.resize(task, height=taskbar_left.shape[0])
        task = cv2.cvtColor(task, cv2.COLOR_RGB2RGBA)
        
        coinflip = random.randint(0, 3)
        if coinflip == 0:
            coinflip = random.randint(0, 0)
            if coinflip == 0:
                #selected_overlay = cv2.resize(taskbar_selected, (task.shape[1], task.shape[0]))
                #task = overlayTransparentImage(task, selected_overlay, 0, selected_overlay.shape[1], 0, selected_overlay.shape[0])
                pass
            
            dot_overlay = imutils.resize(taskbar_dot, width=task.shape[1])
            task = insertTransparentImage(task, dot_overlay, 0, dot_overlay.shape[1], task.shape[0] - dot_overlay.shape[0],  task.shape[0])
        
        task = cv2.cvtColor(task, cv2.COLOR_RGBA2RGB)
        taskbar_left = np.hstack((taskbar_left, task))
    '''
    
    # Add task icons
    for i in range(0, len(tasks)):
        # Load icon
        task = cv2.imread(os.path.join(path_to_icons, tasks[i]), cv2.IMREAD_UNCHANGED)
        
        # Resize icon such that it doesn't fill whole bar
        task = imutils.resize(task, height=int(taskbar_left.shape[0] * 0.75))
        
        # Create transparent task background (which has correct size)
        task_background = np.ones((taskbar_left.shape[0], 60, 4))
        
        # Compute center to place the icon
        x1 = int((task_background.shape[1] - task.shape[1]) / 2)
        x2 = x1 + task.shape[1]
        
        y1 = int((task_background.shape[0] - task.shape[0]) / 2)
        y2 = y1 + task.shape[0]
        
        # Add task icon to transparent background with correct size
        task_background[y1:y2, x1:x2] = task
        task = task_background
        
        # Get the black background
        background = taskbar_middle[:, 0:task.shape[1]].copy()
        
        # Randomly decide if the task is open
        coinflip = random.randint(0, 3)
        if coinflip == 0:
            # Randomly decide if the task is even currently shown
            coinflip = random.randint(0, 3)
            if coinflip == 0:
                # Add "currently shown" effect (gray background)
                selected_overlay = cv2.resize(taskbar_selected, (task.shape[1], task.shape[0]))
                background = insertTransparentImage(background, selected_overlay, 0, selected_overlay.shape[1], 0, selected_overlay.shape[0])
            
            # Add "Open" effect (blue line at bottom)
            dot_overlay = imutils.resize(taskbar_dot, width=task.shape[1])
            background = insertTransparentImage(background, dot_overlay, 0, dot_overlay.shape[1], task.shape[0] - dot_overlay.shape[0],  task.shape[0])
        
        # Add icon to the background
        background = insertTransparentImage(background, task, 0, task.shape[1], 0,  task.shape[0])
        background = cv2.cvtColor(background, cv2.COLOR_RGBA2RGB)
        
        # Append icon to the taskbar (left side)
        taskbar_left = np.hstack((taskbar_left, background))
    
    # Insert left part of taskbar to the wallpaper
    x_offset= 0
    y_offset= wallpaper.shape[0] - taskbar_left.shape[0]
    wallpaper[y_offset:y_offset+taskbar_left.shape[0], x_offset:x_offset+taskbar_left.shape[1]] = taskbar_left

    # Pick a random number of icons for the right part of the tas bar
    number_of_elements = random.randint(1, len(os.listdir(path_to_right_taskbar_elements)))
    tasks = random.sample(os.listdir(path_to_right_taskbar_elements), k=number_of_elements)

    # Add icons to right side of taskbar
    for i in range(0, len(tasks)):
        task = cv2.imread(os.path.join(path_to_right_taskbar_elements, tasks[i]))
        task = imutils.resize(task, height=taskbar_right.shape[0])

        # Append icon to task bar (right side)
        taskbar_right = np.hstack((task, taskbar_right))

    # Insert right part of taskbar to the wallpaper
    x_offset= wallpaper.shape[1] - taskbar_right.shape[1]
    y_offset= wallpaper.shape[0] - taskbar_right.shape[0]
    wallpaper[y_offset:y_offset+taskbar_right.shape[0], x_offset:x_offset+taskbar_right.shape[1]] = taskbar_right

    # Fill the middle part between left and right taskbar with the filler image
    # Compute remaining filler size
    fillwidth = wallpaper.shape[1] - taskbar_right.shape[1] - taskbar_left.shape[1]
    
    # Get y-coordinate
    y_offset= wallpaper.shape[0] - taskbar_middle.shape[0]

    # If there is still space we need to fill, then... (filler image is not infinite, so we may need to fill up multiple times)
    if fillwidth > 0:
        i = 0
        while i < fillwidth:
            # Get x-coordinate
            x_offset = taskbar_left.shape[1] + i
            
            # Cut image if whole image would be larger than fillwidth
            width = min(taskbar_middle.shape[1], fillwidth - i)

            # Add filler image
            wallpaper[y_offset:y_offset+taskbar_middle.shape[0], x_offset:x_offset+width] = taskbar_middle[:, 0:width]

            # Update fillwidth
            i += taskbar_middle.shape[1]
        
    return wallpaper, wallpaper_file

In [19]:
# Show empty desktop image
desktop, _ = createDesktop()

cv2.imshow("Empty Desktop Image", desktop)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
cv2.imwrite(TEST_OUTPUT + str(int(time.time()))+ ".png", desktop)

True

In [20]:
# Create a random window head for a given width
def generateWindowHead(final_width):
    # Get a random application icon
    icon_name = random.choice(os.listdir(path_to_icons))

    # Load and resize icon to window height
    icon = cv2.imread(os.path.join(path_to_icons, icon_name), cv2.IMREAD_UNCHANGED)
    icon = imutils.resize(icon, height=window_end.shape[0])

    # Get background for icon
    image = window_filler.copy()
    image = image[:, 0:icon.shape[1]]
    image = cv2.cvtColor(image, cv2.COLOR_RGB2RGBA)
    
    # Make icon again smaller such that it doesn't fill whole height
    icon = imutils.resize(icon, height=int(window_end.shape[0] * 0.65))
    
    # Center icon
    x1 = int((image.shape[1] - icon.shape[1]) / 2)
    x2 = x1 + icon.shape[1]
    
    y1 = int((image.shape[0] - icon.shape[0]) / 2)
    y2 = y1 + icon.shape[0]
    
    # Add icon to background image
    start = insertTransparentImage(image, icon, x1, x2, y1, y2)
    
    # This is the start of our application head (icon is on left end)
    start = cv2.cvtColor(start, cv2.COLOR_RGBA2RGB)

    # Our title vocabulary
    words = ["Lorem", "Ipsum", "Dolor", "Consetetur", "Sadipscing", "Elitr", "Nonumy", "Eirmod", "Tempor", "Invidunt",
             "Labore", "Dolore", "Magna", "Aliquyam", "Erat", "Voluptua", "Vero", "Accusam", "Dolores", "Rebum"]

    # Select a random title
    text = " ".join(random.sample(words, k=random.randint(1,3)))

    # Get background image for title text
    textImage = window_filler.copy()

    # Get font and font size for title text
    fontsize = int(textImage.shape[0] / 1.8)
    font = ImageFont.truetype(path_to_font_normal, fontsize)  

    # Get text printed on background
    textImage = createTextField(textImage, text, font)
    
    # Append title to the icon
    start = np.hstack((start, textImage))
    
    # Fill remaining width with filler image
    fillwidth = final_width - start.shape[1] - window_end.shape[1]
    y_offset = 0

    if fillwidth > 0:
        # There is still space to fill, so fill
        i = 0
        while i < fillwidth:
            # Cut image if whole image would be larger than fillwidth
            width = min(window_filler.shape[1], fillwidth - i)
            
            # Append filler image
            start = np.hstack((start, window_filler[:, 0:width]))
            
            # Update remaining fillwidth
            i += window_filler.shape[1]
    else:
        # Application head is already too large. Cut such that at least left end still fits to it
        start = start[:, 0:(final_width - window_end.shape[1])]
    
    # Add right end of window head (close window, etc...)
    start = np.hstack((start, window_end))
    
    return start

# Create a random browser head for a given width
def generateBrowserHead(final_width):
    start = edge_start.copy()
    
    # Get a random URL
    words = ["lorem", "ipsum", "dolor", "sit", "amet", "consetetur", "sadipscing", "elitr", "sed", 
             "diam", "nonumy", "eirmod", "tempor", "invidunt", "ut", "labore", "et", "dolore", "magna", 
             "aliquyam", "erat", "sed", "diam", "voluptua", "at", "vero", "eos", "et", "accusam", "et", 
             "justo", "duo", "dolores", "rebum"]

    # Seperator characters which connect the random URL words
    separators = ["/", ".", "-", "_", "=", "+", ""]
    
    # Create random URL
    samples = random.sample(words, k=random.randint(3,15))
    text = ""
    
    # Add random seperators between words
    for s in samples:
        sep = random.choice(separators)
        text += s
        text += sep
    
    # Get background for URL
    textImage = edge_middle.copy() 
    
    # Get URL font
    fontsize = int(textImage.shape[0] / 2.5)
    font = ImageFont.truetype(path_to_font_normal, fontsize) 

    # Add URL text to background
    textImage = createTextField(textImage, text, font)
    
    # Append URL text to start image of browser head
    start = np.hstack((start, textImage))
    
    # Fill remaining width with filler image
    fillwidth = final_width - start.shape[1] - edge_end.shape[1]
    y_offset = 0

    if fillwidth > 0:
        # There is still space to fill, so fill
        i = 0
        while i < fillwidth:
            # Cut image if whole image would be larger than fillwidth
            width = min(edge_middle.shape[1], fillwidth - i)
            
            # Append filler image
            start = np.hstack((start, edge_middle[:, 0:width]))
            
            # Update remaining fillwidth
            i += edge_middle.shape[1]
    else:
        # Browser head is already too large. Cut such that at least left end still fits to it
        start = start[:, 0:(final_width - edge_end.shape[1])]
    
    # Add right end of browser head
    start = np.hstack((start, edge_end))
    
    return start

In [21]:
# Add a random browser window to the desktop screenshot
def addBrowerWindow(screen, website_name, fullscreen):
    # Captures whether we take the large or small version of a website
    version = "large"
    
    # If application is not shown in fullscreen, randomly decide if we go for small or large version
    if not fullscreen:
        coinflip = random.randint(0, 1)
        
        if coinflip == 0:
            version = "small"
    
    # Load website screenshot
    website_path = os.path.join(path_to_websites, version, website_name)
    website = cv2.imread(website_path)
    
    if website is None:
        print(website_path)
    
    # Resize website such that it is not lager than the screenshot
    website = imutils.resize(website, width=min(screen.shape[1] - border_effect_size * 2, website.shape[1]))
    website = imutils.resize(website, height=min(screen.shape[0] - taskbar_left_original.shape[0] - window_end.shape[0] - edge_end.shape[0] - border_effect_size * 2, website.shape[0]))

    # Disable fullscreen mode for webpages which would be rescaled too large
    if website.shape[0] < screen.shape[0] / 2 or website.shape[1] < screen.shape[1] / 2:
        #print(website_path, " is too small for fullscreen mode!")
        fullscreen = False
    
    if fullscreen:
        # Resize webpage to fullscreen
        website = cv2.resize(website, (screen.shape[1], screen.shape[0] - taskbar_left_original.shape[0] - window_end.shape[0] - edge_end.shape[0]))
    else:
        # Randomly scale the webpage, since it's not in fullscreen
        scale = random.uniform(0.7, 1.0)
        website = imutils.resize(website, width=int(website.shape[1] * scale))
    
    # Generate a random application and browser head for webpage
    head = generateWindowHead(website.shape[1])
    browserHead = generateBrowserHead(website.shape[1])

    website = np.vstack((head, browserHead, website))
    website = cv2.cvtColor(website, cv2.COLOR_RGB2RGBA)
    
    # Add window effect to webpage (if not in fullscreen)
    if not fullscreen:
        website = addWindowBorderEffect(website)
    
    # Choose random position for webpage
    x1 = random.randint(0, screen.shape[1] - website.shape[1])
    x2 = x1 + website.shape[1]
    
    y1 = random.randint(0, screen.shape[0] - website.shape[0] - taskbar_left_original.shape[0])
    y2 = y1 + website.shape[0]

    # Add webpage to screenshot
    screen = insertTransparentImage(screen, website, x1, x2, y1, y2)
    
    data = {"name": website_path, "fullscreen": fullscreen, "x1": x1, "x2": x2, "y1": y1, "y2": y2}
    
    return screen, data

# Add a random application window to the desktop screenshot
def addApplicationWindow(screen, app_path, fullscreen):
    # Load application screenshot
    app = cv2.imread(app_path)
    
    # Check if we need to add a application head to screenshot, of whether it already contains one
    additional_offset = 0
    if "_x" not in app_path:
        additional_offset = window_end.shape[0]
    
    # Resize application such that it is not lager than the screenshot
    app = imutils.resize(app, width=min(screen.shape[1] - border_effect_size * 2, app.shape[1]))
    app = imutils.resize(app, height=min(screen.shape[0] - taskbar_left_original.shape[0] - additional_offset - border_effect_size * 2, app.shape[0]))
    
    # Disable fullscreen mode for applications which would be rescaled too large
    if app.shape[0] < screen.shape[0] / 1.5 or app.shape[1] < screen.shape[1] / 1.5:
        #print(app_path, " is too small for fullscreen mode!")
        fullscreen = False
        
    if fullscreen:
        # Resize application to fullscreen
        app = cv2.resize(app, (screen.shape[1], screen.shape[0] - taskbar_left_original.shape[0] - additional_offset))
    else:
        # Randomly scale the application, since it's not in fullscreen
        scale = random.uniform(0.7, 1.0)
        app = imutils.resize(app, width=int(app.shape[1] * scale))
    
    # Generate a random application head if necessary
    if additional_offset > 0:
        head = generateWindowHead(app.shape[1])
        app = np.vstack((head, app))
    
    app = cv2.cvtColor(app, cv2.COLOR_RGB2RGBA)

    # Add window effect to application (if not in fullscreen)
    if not fullscreen:
        app = addWindowBorderEffect(app)
    
    # Choose random position for application
    x1 = random.randint(0, screen.shape[1] - app.shape[1])
    x2 = x1 + app.shape[1]
    
    y1 = random.randint(0, screen.shape[0] - app.shape[0] - taskbar_left_original.shape[0])
    y2 = y1 + app.shape[0]

    # Add application to screenshot
    screen = insertTransparentImage(screen, app, x1, x2, y1, y2)
    
    data = {"name": app_path, "fullscreen": fullscreen, "x1": x1, "x2": x2, "y1": y1, "y2": y2}
    
    return screen, data

In [22]:
# Create a random desktop shortcut
def createShortcut(icon_name, icon_size):
    # Vocabulary for shortcut text
    words = ["lorem", "ipsum", "dolor", "sit", "amet", "consetetur", "sadipscing", "elitr", "sed", 
         "diam", "nonumy", "eirmod", "tempor", "invidunt", "ut", "labore", "et", "dolore", "magna", 
         "aliquyam", "erat", "sed", "diam", "voluptua", "at", "vero", "eos", "et", "accusam", "et", 
         "justo", "duo", "dolores", "rebum"]

    # Padding of the shortcut
    icon_within_border = int(icon_size * 0.26)

    # Height of the icon
    icon_image_max_height = int(icon_size * 0.8) - icon_within_border
    
    # Height of the shortcut text
    icon_text_max_height = icon_size - icon_image_max_height - icon_within_border

    # Get the shortcut font
    fontsize = int(icon_within_border / 2.0)
    font = ImageFont.truetype(path_to_font_normal, fontsize)
    
    # Load the shortcut icon
    icon = cv2.imread(os.path.join(path_to_icons, icon_name), cv2.IMREAD_UNCHANGED)

    # Max possible width of icon
    max_width = min(icon_size - 2*icon_within_border, icon.shape[1])
    icon = imutils.resize(icon, width=max_width)

    # Max possible height of icon
    max_height = min(icon_image_max_height, icon.shape[0])
    icon = imutils.resize(icon, height=max_height)

    # Center icon 
    x = int((icon_size - icon.shape[1]) / 2)
    y = icon_within_border + int((icon_image_max_height - icon.shape[0]) / 2)

    # Add the icon
    image = np.zeros((icon_size, icon_size, 4), np.uint8)
    image[y:y+icon.shape[0], x:x+icon.shape[1]] = icon

    # Generate the random text
    text = " ".join(random.choices(words, k=random.randint(1,3)))
    
    # Add text
    image = addShortcutText(image, text, font, icon_within_border, icon_text_max_height, icon_image_max_height)
    
    return image

In [25]:
# Fills the desktop screenshot with content
def fillDesktop(screen):
    # ==> Desktop Icons:

    # Define shortcut size
    icon_size = int(taskbar_left_original.shape[0] * 2.2)
    
    # Define additional (horizontal) margin between shortcuts
    icon_outside_border = int(icon_size * 0.05)
    
    # Compute grid size for shortcuts
    row_size = int((screen.shape[0] - taskbar_left_original.shape[0]) / icon_size)
    col_size = int((screen.shape[1] - icon_outside_border) / (icon_size + icon_outside_border))
    
    # Enlarge horizontal margin between shortcuts such that there is no potential large gap at end
    rest_outside_border = screen.shape[1] - (col_size * icon_size + (col_size+1) * icon_outside_border)
    icon_outside_border += int(rest_outside_border / (col_size + 1))
    
    # Chance that there is a block of shortcuts at the border of the screen
    # The larger the number, the less likely
    border_chance = random.randint(5, 10)
    
    # Chance that there is a block of shortcuts within the screen
    normal_chance = random.randint(border_chance*2, border_chance*4)
    
    # Grid matrix which hold where a shortcut is placed (1 shortcut, 0 no shortcut)
    icon_matrix = np.zeros((row_size, col_size))
    
    # Traverse the border of the grid (counter clockwise)
    steps = [[1, 0], [0, 1], [-1, 0], [0, -1]]

    x = 0
    y = 0
    i = 0
    
    while True:
        # Check if we hit a corner, if so, change the traverse direction
        if y + steps[i][0] < 0 or y + steps[i][0] == icon_matrix.shape[0]:
            i += 1
        elif x + steps[i][1] < 0 or x + steps[i][1] == icon_matrix.shape[1]:
            i += 1
        
        # We did one whole turn around the border, stop
        if i == 4:
            break
        
        # Get next grid field
        x += steps[i][1]
        y += steps[i][0]
        
        # If there already is an shortcut, continue
        if icon_matrix[y, x] == 1:
            continue
        
        # Decide if there is a shortcut
        coinflip = random.randint(0, border_chance)
                
        if coinflip == 0:
            # We place a shortcut. Now decide how many surrounding shortcuts there are
            
            # Vertical direction
            if steps[i][0] == 1 or (steps[i][0] == 0 and y == 0):
                # We are at left border (going down) or at the top border (going left)
                y1 = y
                y2 = y + random.randint(0, min(2, icon_matrix.shape[0] - y - 1)) + 1
            else:
                y1 = y - random.randint(0, min(2, y))
                y2 = y + 1
            
            # Horizontal direction
            if steps[i][1] == 1 or (steps[i][1] == 0 and x == 0):
                # We are at bottom border (going right) or at the left border (going down)
                x1 = x
                x2 = x + random.randint(0, min(4, icon_matrix.shape[1] - x - 1)) + 1
            else:
                x1 = x - random.randint(0, min(4, x))
                x2 = x + 1
            
            # Fill with 1s (i.e., add block of shortcuts)
            icon_matrix[y1:y2, x1:x2] = 1
    
    # Select a random number of desktop icons which will be added to the left side of screen (one.after.the.other)
    number_of_icons = random.randint(6, 25)
    for i in range(0, number_of_icons):
        row = int(i / row_size)
        col = i % row_size
        
        icon_matrix[col, row] = 1
        
    # Decide for shortcut blocks within the screenshot (not border)
    for y in range(0, icon_matrix.shape[0]):
        for x in range(0, icon_matrix.shape[1]):
            if icon_matrix[y, x] == 1:
                continue
            
            # Randomly decide if we add a shortcut
            coinflip = random.randint(0, normal_chance)
            
            if coinflip == 0:
                # Decide for size of shortcut block
                height = y + random.randint(1, min(2, icon_matrix.shape[0] - y))
                width = x + random.randint(1, min(3, icon_matrix.shape[1] - x))
                
                icon_matrix[y:height, x:width] = 1
                
    #print(icon_matrix)

    # Draw shortcuts to the screenshot
    icon_files = os.listdir(path_to_icons)
    
    for i in range(0, row_size):
        for j in range(0, col_size):
            if icon_matrix[i, j] == 1:
                # Select random icon
                icon_name = random.choice(icon_files)
                image = createShortcut(icon_name, icon_size)

                # Compute position
                x1 = j * icon_size + (j+1) * icon_outside_border
                x2 = x1 + image.shape[1]

                y1 = i * icon_size
                y2 = y1 + image.shape[0]
                
                # Draw on screenshot
                screen = insertTransparentImage(screen, image, x1, x2, y1, y2)

                
    # ==> Application Windows:
    applications = []
    
    # Decide for a random number of application windows
    number_of_applications = random.randint(0, 5)
    
    # Load all application windows form dataset
    for d in os.listdir(path_to_screenshots):
        new_path = os.path.join(path_to_screenshots, d)
        
        if os.path.isdir(new_path):
            for img in os.listdir(new_path):
                applications.append(os.path.join(new_path, img))
    
    # Sample random screenshots
    app_paths = []
    if len(applications) >= number_of_applications:
        # Only do this if we have provided enough application screenshots
        app_paths = random.sample(applications, k=number_of_applications)

    # Load webpage screenshots from dataset
    websites = os.listdir(os.path.join(path_to_websites, "large"))
    
    # Sample random webpages
    website_names = random.sample(websites, k=number_of_applications)

    data = []
    
    # Draw windows
    for i in range(0, number_of_applications):
        coinflip = random.randint(0, 4)
    
        # Decide if fullscreen or not
        if coinflip == 0:
            fullscreen = True
        else:
            fullscreen = False
        
        # Decide if webpage or application 
        coinflip = random.randint(0, 2)
        
        if coinflip == 0 and len(app_paths) > 0:
            screen, d = addApplicationWindow(screen, app_paths[i], fullscreen)
        else:
            screen, d = addBrowerWindow(screen, website_names[i], fullscreen)
            
        data.append(d)

    # ==> Mouse Icon:

    # Get a random position for mouse icon
    x1 = random.randint(0, screen.shape[1] - mouse_icon.shape[1])
    x2 = x1 + mouse_icon.shape[1]
    
    y1 = random.randint(0, screen.shape[0] - mouse_icon.shape[0])
    y2 = y1 + mouse_icon.shape[0]

    # Draw mouse icon
    #screen = insertTransparentImage(screen, mouse_icon, x1, x2, y1, y2)
    
    return screen, data

In [26]:
# Randomly fill screenshot
screen = desktop.copy()
screen, _ = fillDesktop(screen)

cv2.imshow("Desktop Image", screen)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [60]:
cv2.imwrite(TEST_OUTPUT + str(int(time.time()))+ ".png", screen)

True

In [15]:
# Prints the progress in the console
def print_progress(count, total):
    pct_complete = float(count) / total
    msg = "\r- Progress: {0:.1%}".format(pct_complete)

    # Print it.
    sys.stdout.write(msg)
    sys.stdout.flush()

In [17]:
# Output path of synthesized screenshots
output_path = "C:/Users/Sander/Desktop/Dataset/Out1/Windows/"

if not os.path.exists(output_path):
    os.mkdir(output_path)

# Number of synthesized screenshots
number_of_images = 100

# Create random screenshots
for i in range(0, number_of_images):
    print_progress(i+1, number_of_images)
    image, wallpaperID = createDesktop()
    image, data = fillDesktop(image)
    cv2.imwrite(output_path + str(i+1) + ".png", image)
    
    with open(output_path + str(i+1) + ".json", 'w') as json_file:
        json.dump({"background" : wallpaperID, "applications" : data}, json_file)

print()
print("Terminated")

- Progress: 100.0%
Terminated
